# Galaxy Selector MOC

**Compared to the "Simple Galaxy Selector, this works with Multi-Ordering-Sky-Maps**

**ASTROPY_HEALPIX NOT INSTALLED -> DOES NOT WORK ON NERSC**


***NOTE (this note may be wrong, just copied from old notebook)***: Before anything can happen, download the GW event localization map. Open the terminal and type:
"curl -O https://gracedb.ligo.org/api/superevents/sid/files/bayestar.fits,0"
with sid = superevent ID (could be incorporated into this notebook)

Select the galaxies to observe by WWFI in this simple manner:
- get the LIGO event data
- read the LIGO event data into the notebook, extract some information
- read the DESI database into the notebook, "clean" the data
- get the 90% credible regions for the event, add them as a column to the data
- only keep data within the 90% region
- calculate all the luminosities (& more)
- rank them by luminosity (for now just print the TARGETID), 3D/2D localization and with a luminosity-distance 2D dependant counterpart likelihood

Sources: 
- https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10
- https://emfollow.docs.ligo.org/userguide/tutorial/skymaps.html
- https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html
- https://arxiv.org/pdf/1710.05452.pdf

In [10]:
import numpy as np
from scipy.stats import norm
import scipy as sc

from astropy.io import fits
from astropy.table import Table, hstack
from astropy import table

import healpy as hp
import astropy_healpix as ah

from astropy.table import QTable

import psycopg2

from astropy.cosmology import Planck18, z_at_value
from astropy.coordinates import Distance 
from astropy import units as u

import astropy.constants as asc

import matplotlib.pyplot as plt

Specifiy the path where the data is located and then load the data (this loads everything, just have a look)

In [3]:
path = "/global/homes/j/jgassert/Code/data/bayestar.multiorderS200311bg.fits,1"
skymap = QTable.read(path)

skymap[:5]

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
1024,2.810789289065552e-31,368.45220053665895,203.98712615523553,5.651336738354135e-06
1025,2.2410403428136984e-31,inf,1.0,0.0
1026,2.0339165245847635e-30,468.54148826812605,94.01334803516339,4.378865855578356e-06
1027,1.742573180996848e-31,-693.5010210710931,584.6367464295139,5.9324972315106206e-05
1028,5.0178196630906625e-30,inf,1.0,0.0


### Get basic values from GW event

We now extract some basic information  from the healpix data:
- ipix_max: pixel with highest likelihood
- npix: total number of pixels
- ra, dec: ra and dec of the pixel with the highest likelihood

Then we open the full fits file and extract some more basic info from the header:
- dist_mean: mean distance of the GW event
- dist_std: error of the distance

In [4]:
# most probable sky location
i = np.argmax(skymap["PROBDENSITY"])
uniq = skymap[i]["UNIQ"]
print("UNIQ ID of highest prob sky localization: ", uniq)

UNIQ ID of highest prob sky localization:  8635874


In [5]:
# calculate the most probable pixel, convert to RA and dec
level_max, ipix_max = ah.uniq_to_level_ipix(uniq)
nside = ah.level_to_nside(level_max)
ra, dec = ah.healpix_to_lonlat(ipix_max, nside, order = "nested")
print("Highest probability at (RA, dec): ", ra.deg, dec.deg, "with prob [1/deg^2] of: ", skymap[i]['PROBDENSITY'].to_value(u.deg**-2))

Highest probability at (RA, dec):  1.8896484375000004 -6.955230191010185 with prob [1/deg^2] of:  0.04597608759643093


Now open the fits file and read basics info from the header

In [6]:
fits_gw = fits.open(path)
header = fits_gw[1].header
dist_mean = header["DISTMEAN"]
dist_std = header["DISTSTD"]
print("The dist_mean and dist_std values: ", dist_mean*u.Mpc, dist_std*u.Mpc)

The dist_mean and dist_std values:  924.038047662503 Mpc 188.4869314505396 Mpc


In [14]:
# these values (924, 188Mpc) are close to z= 0.19, 0.03

### Get DESI data

Establish a connection to the DESI database and load the daily data.

In [6]:
try:
    db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi', password = "5kFibers!", port="5432")
    cursor = db.cursor()
except (Exception, psycopg2.Error) as error:
    print(error)

cursor = db.cursor()

In [8]:
redux = 'daily'
query = 'SELECT f.targetid,f.target_ra,f.target_dec,c.tileid,c.night,r.z,r.zerr,r.zwarn,r.deltachi2,f.flux_z,f.bgs_target,f.ebv, f.sersic, f.mws_target, c.filename\n' \
                    f'FROM {redux}.tiles_fibermap f\n' \
                    f'INNER JOIN {redux}.cumulative_tiles c ON f.cumultile_id=c.id\n' \
                    f'INNER JOIN {redux}.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid\n' \
                    f'WHERE q3c_radial_query( f.target_ra, f.target_dec, {ra.deg}, {dec.deg}, 50);'

cursor.execute(query)
rows = cursor.fetchall()

## Do all the calculations, selections,...

- turn the daily DESI data into a Table
- do some basic data selection stuff: only keep good + positive redshifts, only positive values of z-band flux and eliminate duplicates (right now simply takes the first entry)
- add the ipix (healpix) pixel value for each target
- calculate the probability for each target
- calculate the probabilities for each object
- add these values to the able ("PROB"), select only targets within the 90% credible region (i.e. with a certainty of 90% the GW event is inside this region)
- calculate the distances and its errors from the redshift, add to the data table
- calculate the 3D probability just like here: https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10 (§4) and add these values to the table
- sort the table by its 3D probability (descending)
- calculate absolute and apparent magnitude, luminosity and add to table

The final step is to include the calculation of the most likely host galaxy from https://arxiv.org/pdf/1710.05452.pdf (GW170817); these values are then also added to the table as "P_GAL"

In [199]:
if rows:
    data = Table(list(map(list, zip(*rows))),
                             names=['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'TILEID', 'NIGHT', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FLUX_Z', 'BGS_TARGET', 'EBV', 'SERSIC', 'MWS_TARGET','FILENAME'])
data[:5:-1] 

TARGETID,TARGET_RA,TARGET_DEC,TILEID,NIGHT,Z,ZERR,ZWARN,DELTACHI2,FLUX_Z,BGS_TARGET,EBV,SERSIC,MWS_TARGET,FILENAME
int64,float64,float64,int64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,str71
2305843041975433202,237.139251601027,-6.16722128539336,40020,20230501,0.000488501923668986,6.7198750610656e-07,0,207215.970368864,-99.0,0,0.159462139010429,0.0,1152921504606846976,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041975438825,237.351657790797,-5.94973101861982,40020,20230501,2.11552184318834e-05,3.52892141575608e-07,0,886573.461774786,-99.0,0,0.188542142510414,0.0,1152921504606846976,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
-400209204,237.201722331555,-6.07920818086159,40020,20230501,0.00129190278695159,9.34367337042744e-05,1,13.1656401884939,0.0,0,0.160429581999779,0.0,0,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
-400209218,237.27435919995,-6.08513032789981,40020,20230501,1.48812250211619,0.000184930904625776,1,28.405016079545,0.0,0,0.170396596193314,0.0,0,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041975432625,237.182030703466,-6.30925426080948,40020,20230501,-0.000174688182812153,4.01365425623457e-06,0,5584.83682483853,-99.0,0,0.164259031414986,0.0,288230376151711744,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041975435467,237.242614089092,-6.09027455473551,40020,20230501,0.000207226016957094,6.11075181929302e-06,0,1537.40508953569,-99.0,0,0.164962008595467,0.0,288230376151711744,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041988016046,237.70574697114,-5.92466221631669,40020,20230501,-0.000269137994562703,1.21995786441489e-06,0,54062.0109465284,-99.0,0,0.213628515601158,0.0,1152921504606846976,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041975402913,236.962267172561,-5.63283633588159,40020,20230501,0.000192521901830522,2.05681943248791e-06,0,20297.0332030501,-99.0,0,0.191328108310699,0.0,2305843009213693952,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits
2305843041979612532,237.721708970586,-5.97028782087754,40020,20230501,-0.000531061403156393,1.02161353418203e-06,0,75416.411093223,-99.0,0,0.214854791760445,0.0,1152921504606846976,daily/tiles/cumulative/40020/20230501/redrock-9-40020-thru20230501.fits


basic data clean up:

In [200]:
data = data[data['ZWARN']==0]
data = data[data['Z']>=0]
data = data[data['FLUX_Z']>0]
data = table.unique(data, keys = "TARGETID")

calculate the healpix id for every object in the catalog and its probability; then append this data to the Table

In [201]:
max_level = 29 # highest possible HEALPix resolution that can be represented in a 64-bit signed integer
max_nside = ah.level_to_nside(max_level)
level, ipix = ah.uniq_to_level_ipix(skymap["UNIQ"])

In [202]:
level, len(level), ipix, len(ipix) 

(array([ 4,  4,  4, ..., 10, 10, 10]),
 19200,
 array([      0,       1,       2, ..., 4454033, 4454034, 4454035]),
 19200)

In [203]:
index = ipix*(2**(max_level-level))**2

In [204]:
index,len(index)

(array([                  0,    1125899906842624,    2251799813685248, ...,
        1224315268499505152, 1224315543377412096, 1224315818255319040]),
 19200)

In [205]:
sorter = np.argsort(index)

In [206]:
sorter, len(sorter)

(array([   0,    1,    2, ..., 2301, 2302, 2303]), 19200)

In [207]:
match_ipix = ah.lonlat_to_healpix(data["TARGET_RA"]*u.deg, data["TARGET_DEC"]*u.deg, max_nside, order='nested')


In [208]:
match_ipix

array([2728622294581067943, 2728621779287900438, 2728621850916985205, ...,
        727443823644003449,  727441492111965904,  727441547847715670])

In [209]:
i = sorter[np.searchsorted(index, match_ipix, side='right', sorter=sorter) - 1]

In [210]:
i, len(i)

(array([1939, 1939, 1939, ...,  645,  645,  645]), 6792308)

In [211]:
probs = skymap[i]['PROBDENSITY'].to_value(u.deg**-2)

In [212]:
probs, len(probs)

(array([5.71064285e-34, 5.71064285e-34, 5.71064285e-34, ...,
        7.17220113e-33, 7.17220113e-33, 7.17220113e-33]),
 6792308)

In [213]:
len(data)

6792308

In [214]:
#data = data.keep_columns(["TARGETID", "TARGET_RA", "TARGET_DEC", "TILEID", "NIGHT", "Z", "ZERR", "ZWARN", "DELTACHI2", "FLUX_Z", "BGS_TARGET", "EBV", "SERSIC", "MWS_TARGET", "FILENAME"])
                          

In [215]:
data = hstack([data, Table({"UNIQ": i}), Table({"PROB": probs})])

In [216]:
data[:5]

TARGETID,TARGET_RA,TARGET_DEC,TILEID,NIGHT,Z,ZERR,ZWARN,DELTACHI2,FLUX_Z,BGS_TARGET,EBV,SERSIC,MWS_TARGET,FILENAME,UNIQ,PROB
int64,float64,float64,int64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,str71,int64,float64
39627126168619831,161.689448670564,-28.4007256042119,80701,20210212,1.46691685301265e-05,5.74421115972459e-06,0,701.924735742974,0.184977978467941,0,0.0746785253286362,0.0,0,daily/tiles/cumulative/80701/20210212/zbest-1-80701-thru20210212.fits,1939,5.710642848039233e-34
39627126168620535,161.722238626931,-28.4321405784131,80701,20210212,0.81358522705151,5.37247861747279e-06,0,3823.5796546936,0.7922483086586,0,0.0770527869462967,1.0,0,daily/tiles/cumulative/80701/20210212/zbest-1-80701-thru20210212.fits,1939,5.710642848039233e-34
39627126168621116,161.748119776018,-28.4055408577292,80701,20210212,0.874069418159146,2.49326327116228e-05,0,303.452315043658,1.01588177680969,0,0.0731853172183037,1.0,0,daily/tiles/cumulative/80701/20210212/zbest-1-80701-thru20210212.fits,1939,5.710642848039233e-34
39627126168621739,161.7753477084,-28.4655301381113,80701,20210212,0.000318077698234015,6.47856441597133e-05,0,2348.66453922422,9.62630081176758,0,0.0818678736686707,0.0,0,daily/tiles/cumulative/80701/20210212/zbest-1-80701-thru20210212.fits,1939,5.710642848039233e-34
39627126168622209,161.795718217228,-28.389996260381,80701,20210212,1.26755752535543e-05,5.01738484800885e-06,0,764.922629979174,0.830278873443604,0,0.0709665864706039,1.0,0,daily/tiles/cumulative/80701/20210212/zbest-1-80701-thru20210212.fits,1939,5.710642848039233e-34


In [217]:
i[0]

1939

In [218]:
np.min(probs), np.max(probs)

(3.169267167263445e-39, 5.304415237478843e-30)

In [219]:
type(skymap)

astropy.table.table.QTable

In [220]:
skymap_sort = Table(skymap, copy = True)

In [221]:
# find the 90% credible region

skymap_sort.sort('PROBDENSITY', reverse=True)

In [222]:
skymap

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
1024,2.810789289065552e-31,368.45220053665895,203.98712615523553,5.651336738354135e-06
1025,2.2410403428136984e-31,inf,1.0,0.0
1026,2.0339165245847635e-30,468.54148826812605,94.01334803516339,4.378865855578356e-06
1027,1.742573180996848e-31,-693.5010210710931,584.6367464295139,5.9324972315106206e-05
1028,5.0178196630906625e-30,inf,1.0,0.0
1029,4.797580236086145e-29,-3413.159103252367,1168.6465400854165,0.0026780709901027743
1030,1.9570653020315176e-30,-683.136170177018,648.5468293392516,3.554211921364464e-05
1031,2.1886313614548153e-29,-382.0219944728645,580.2976827897337,1.9247360186695577e-05


In [223]:
skymap_sort

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
8635874,150.9305923102596,864.0544123393728,177.93280764677917,1.2849339652181568e-06
8635829,150.90015925644565,864.2823428505719,177.80232609133046,1.2843604402415653e-06
8635881,150.81080736723487,863.8272214134608,178.0667315631715,1.2855036372743902e-06
8635806,150.7196217659106,864.5086898474821,177.67590777910323,1.2837893324140424e-06
8636263,150.6357713408702,868.0486892889259,174.41913767344835,1.2756231847675179e-06
8636280,150.60578927581113,867.7820718105779,174.582871711354,1.2762836212810563e-06
8635880,150.5516562424454,865.1611633727159,177.40748819509628,1.282088669267722e-06
8635886,150.53933218598846,863.5978628495068,178.2052621826929,1.2860770614342774e-06


In [224]:
level, ipix = ah.uniq_to_level_ipix(skymap_sort['UNIQ'])
pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level))

In [225]:
prob = pixel_area * skymap_sort['PROBDENSITY']

In [226]:
cumprob = np.cumsum(prob)

In [227]:
i = cumprob.searchsorted(0.9)

In [228]:
i

7896

In [229]:
area_90 = pixel_area[:i].sum()
area_90.to_value(u.deg**2)

51.97063618706022

In [230]:
skymap_sort[:i]

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
8635874,150.9305923102596,864.0544123393728,177.93280764677917,1.2849339652181568e-06
8635829,150.90015925644565,864.2823428505719,177.80232609133046,1.2843604402415653e-06
8635881,150.81080736723487,863.8272214134608,178.0667315631715,1.2855036372743902e-06
8635806,150.7196217659106,864.5086898474821,177.67590777910323,1.2837893324140424e-06
8636263,150.6357713408702,868.0486892889259,174.41913767344835,1.2756231847675179e-06
8636280,150.60578927581113,867.7820718105779,174.582871711354,1.2762836212810563e-06
8635880,150.5516562424454,865.1611633727159,177.40748819509628,1.282088669267722e-06
8635886,150.53933218598846,863.5978628495068,178.2052621826929,1.2860770614342774e-06


In [240]:
mask = [data["UNIQ"][q] in skymap_sort["UNIQ"][:i] for q in range(len(data))]

In [239]:
len(data)

6792308

In [246]:
np.max(data["UNIQ"]), np.min(skymap_sort["UNIQ"])

(2268, 1024)

In [248]:
1024 in data["UNIQ"]

False

In [241]:
data_90 = data[mask]

In [242]:
data_90[:5]

TARGETID,TARGET_RA,TARGET_DEC,TILEID,NIGHT,Z,ZERR,ZWARN,DELTACHI2,FLUX_Z,BGS_TARGET,EBV,SERSIC,MWS_TARGET,FILENAME,UNIQ,PROB
int64,float64,float64,int64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,str71,int64,float64


In [243]:
distances = Distance(z=prob_data["Z"], cosmology=Planck18)
distances_err = Distance(z=prob_data["ZERR"], cosmology=Planck18)

NameError: name 'prob_data' is not defined

In [ ]:
dP_dV = data_90["IPIX"]*distnorm[data_90["IPIX"]]*norm(distmu[data_90["IPIX"]], distsigma[data_90["IPIX"]]).pdf(distances)/pixarea